In [1]:
import os
import sys
sys.path.insert(0, "..")
import inspect
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import wandb
from safetensors import safe_open
from tqdm import tqdm

import matplotlib.pyplot as plt
# from notebook_utils import *

from models import *
from my_datasets import *
from experiments import *
from experiments.utils.model_loader_utils import *

In [24]:
n = 16
_, res_dataset = load_model_and_dataset_from_big_grid(2*n, n)
model = TheoryAutoregKStepsModel(num_vars=n, num_steps=3)

Querying id: model-SynSAR_gpt2_d32_L1_H1__DAS_nv16_ns3_nr16-32_exph4.000_cl3-5__ntr524288_ntt65536_bsz512_steps16384_lr0.00050_seed571:v0
Downloading: <Artifact QXJ0aWZhY3Q6ODA4NjAwNDM3>


wandb:   2 of 2 files downloaded.  


In [25]:
atk_dataset = KnowledgeAmnesiaDataset(res_dataset, dataset_len = 1024)
dataloader = DataLoader(atk_dataset, batch_size=3)

In [26]:
for batch in dataloader:
    tokens = batch["tokens"]
    abcde = batch["abcde"]
    adv_labels = batch["labels"]
    break

In [27]:
a, b, c, d, e = abcde.chunk(5, dim=-1)

In [35]:
atk_rule = torch.cat([F.one_hot(a,n), -1000*F.one_hot(a,n)], dim=-1).view(-1,1,2*n)

In [36]:
adv_tokens = torch.cat([atk_rule, tokens], dim=1)

In [37]:
adv_out = model(tokens=adv_tokens)

In [38]:
adv_pred = (adv_out.logits > 0).long()

In [39]:
adv_pred

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]]])

In [ ]:
adv_labels = batch["labels"]

In [12]:
atk_rule = 

{'tokens': tensor([[[1., 1., 0.,  ..., 1., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 1., 0.],
          ...,
          [0., 1., 0.,  ..., 0., 0., 1.],
          [0., 1., 1.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[1., 1., 0.,  ..., 1., 0., 1.],
          [1., 1., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 1., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 'labels': tensor([[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 